# Produisez une étude de marché

# Scénario
Votre entreprise **d'agroalimentaire** souhaite se développer à l'international. Elle est spécialisée dans...
 le poulet !

L'international, oui, mais pour l'instant, le champ des possibles est bien large : aucun pays particulier ni aucun continent n'est pour le moment choisi. Tous les pays sont envisageables !

Votre objectif sera d'aider **à cibler plus particulièrement certains pays**, dans le but d'approfondir ensuite l'étude de marché. Plus particulièrement, l'idéal serait de produire des "groupes" de pays, plus ou moins gros, dont on connaît les caractéristiques.

Dans un premier temps, la stratégie est plutôt d'exporter les produits plutôt que de produire sur place, c'est-à-dire dans le(s) nouveau(x) pays ciblé(s).


Pour identifier les pays propices à une insertion dans le marché du poulet,
 Il vous a été demandé de cibler les pays. 
 Etudier les régimes alimentaires de chaque pays, notamment en termes de protéines d'origine animale et en termes de calories.


Construisez votre échantillon contenant l'ensemble des pays disponibles, chacun caractérisé par ces variables :

différence de population entre une année antérieure (au choix) et l'année courante, exprimée en pourcentage ;
proportion de protéines d'origine animale par rapport à la quantité totale de protéines dans la disponibilité alimentaire du pays ;
disponibilité alimentaire en protéines par habitant ;
disponibilité alimentaire en calories par habitant.

Construisez un dendrogramme contenant l'ensemble des pays étudiés, puis coupez-le afin d'obtenir 5 groupes.

Caractérisez chacun de ces groupes selon les variables cités précédemment, et facultativement selon d'autres variables que vous jugerez pertinentes (ex : le PIB par habitant). Vous pouvez le faire en calculant la position des centroïdes de chacun des groupes, puis en les commentant et en les critiquant au vu de vos objectifs.

Donnez une courte liste de pays à cibler, en présentant leurs caractéristiques. Un découpage plus précis qu'en 5 groupes peut si besoin être effectué pour cibler un nombre raisonnable de pays. 

Visualisez vos  partitions dans le premier plan factoriel obtenu par ACP.

Dans votre partition, vous avez obtenu des groupes distincts. Vérifiez donc qu'ils diffèrent réellement. Pour cela, réalisez les tests statistiques suivants :

un test d'adéquation : parmi les 4 variables, ou parmi d'autres variables que vous trouverez pertinentes, trouvez une variable dont la loi est normale ;
un test de comparaison de deux populations (dans le cas gaussien) : choisissez 2 clusters parmi ceux que vous aurez déterminé. Sur ces 2 clusters, testez la variable gaussienne grâce à un test de comparaison.

In [1]:
# Librairies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
import sklearn as sk
import scipy as sp
import pca as pca






In [2]:
#Versions utilisées
print("Jupyter Notebook : " + str(pd.__version__))
print("Pandas : " + str(pd.__version__))
print("Numpy : " + str(np.__version__))
print("Seaborn : " + str(sns.__version__))

Jupyter Notebook : 1.2.4
Pandas : 1.2.4
Numpy : 1.20.1
Seaborn : 0.11.1


In [3]:
sns.set( # Styles Seaborn
    style='whitegrid',
    context='notebook',
    palette='Paired',
    rc={'figure.figsize':(8,5)})

In [4]:
# Couleurs
red_a400 = '#ff1744'
indigo_a100 = '#8c9eff'
indigo_a400 = '#3d5afe'
indigo_a700 = '#304ffe'
bluegrey_900 = '#263238'
teal_a400 = '#1de9b6'
purple_a400 = '#d500f9'
yellow_a400 = '#ffea00'
bluegrey_900 = '#263238'
palette_5 = [teal_a400, indigo_a400, red_a400, yellow_a400, purple_a400]

# 1. Import des données

## 1.1 Import et traitement de la population par pays
la variation de la population entre 2013 et 2019, en %, qui sera positive en cas de croissance ou négative en cas de baisse démographique ;

In [5]:
df_population = pd.read_csv('datas/FAOSTAT_data_2008_2018.csv' , header=0, sep=",", decimal=".")
df_population = df_population.rename(columns= {'Code zone (FAO)': 'Code zone'})
df_population.head(2)

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
0,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2008,2008,1000 personnes,27722.276,X,Sources internationales sûres,NaN
1,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2018,2018,1000 personnes,37171.921,X,Sources internationales sûres,NaN


## 2.1 Données sur les bilans alimentaires mondiaux (2018)


In [23]:
df_dispo_alimentaire = pd.read_csv('datas/FAOSTAT_data_2-16-2022 .csv', header=0, sep=",", decimal=".")
df_dispo_alimentaire.head(4)

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
0,FBS,Bilans Alimentaires (2010-),2,Afghanistan,664,Disponibilité alimentaire (Kcal/personne/jour),2901,Total General,2018,2018,Kcal/personne/jour,2270.00,Fc,Donnée calculée
1,FBS,Bilans Alimentaires (2010-),2,Afghanistan,674,Disponibilité de protéines en quantité (g/pers...,2901,Total General,2018,2018,g/personne/jour,57.62,Fc,Donnée calculée
2,FBS,Bilans Alimentaires (2010-),2,Afghanistan,664,Disponibilité alimentaire (Kcal/personne/jour),2941,Produits Animaux,2018,2018,Kcal/personne/jour,189.00,Fc,Donnée calculée
3,FBS,Bilans Alimentaires (2010-),2,Afghanistan,674,Disponibilité de protéines en quantité (g/pers...,2941,Produits Animaux,2018,2018,g/personne/jour,10.69,Fc,Donnée calculée


## 3.1  les valeurs de PIB par habitant,



In [7]:

df_indicateurs_macro = pd.read_csv('datas/Produit intérieur brut par habitant.csv', header=0, sep=',', decimal='.')
df_indicateurs_macro = df_indicateurs_macro.rename(columns= {'Code zone (FAO)': 'Code zone'})

df_indicateurs_macro.head(2)

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
0,FS,Données de la sécurité alimentaire,2,Afghanistan,6126,Valeur,22013,"PIB par habitant, ($ PPA internationaux consta...",2008,2008,I$,1484.1,X,Sources internationales sûres,NaN
1,FS,Données de la sécurité alimentaire,2,Afghanistan,6126,Valeur,22013,"PIB par habitant, ($ PPA internationaux consta...",2018,2018,I$,2033.8,X,Sources internationales sûres,NaN


## 4.1 Le taux de dépendance aux importations, 


In [8]:
df_dépendance_importations = pd.read_csv('datas/FAOSTAT_data_volaiescsv.csv', header=0, sep=",", decimal=".")
df_dépendance_importations.head(3)

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
0,FBS,Bilans Alimentaires (2010-),2,Afghanistan,5511,Production,2734,Viande de Volailles,2018,2018,Milliers de tonnes,29,Im,Données de la FAO basées sur une méthodologie ...
1,FBS,Bilans Alimentaires (2010-),2,Afghanistan,5611,Importations - Quantité,2734,Viande de Volailles,2018,2018,Milliers de tonnes,25,Im,Données de la FAO basées sur une méthodologie ...
2,FBS,Bilans Alimentaires (2010-),2,Afghanistan,5911,Exportations - Quantité,2734,Viande de Volailles,2018,2018,Milliers de tonnes,0,Im,Données de la FAO basées sur une méthodologie ...


## 5.1 Construction et sauvegarde du dataframe principal


In [9]:
data = pd.concat([df_population, df_dispo_alimentaire, df_dépendance_importations, df_indicateurs_macro]) # Concaténation
data

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
0,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2008,2008,1000 personnes,27722.276,X,Sources internationales sûres,NaN
1,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2018,2018,1000 personnes,37171.921,X,Sources internationales sûres,NaN
2,OA,Séries temporelles annuelles,202,Afrique du Sud,511,Population totale,3010,Population-Estimations,2008,2008,1000 personnes,49779.471,X,Sources internationales sûres,NaN
3,OA,Séries temporelles annuelles,202,Afrique du Sud,511,Population totale,3010,Population-Estimations,2018,2018,1000 personnes,57792.518,X,Sources internationales sûres,NaN
4,OA,Séries temporelles annuelles,3,Albanie,511,Population totale,3010,Population-Estimations,2008,2008,1000 personnes,3002.678,X,Sources internationales sûres,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,FS,Données de la sécurité alimentaire,251,Zambie,6125,Valeur,21032,Stabilité politique et absence de violence/ter...,2018,2018,indice,0.090,X,Sources internationales sûres,NaN
760,FS,Données de la sécurité alimentaire,181,Zimbabwe,6126,Valeur,22013,"PIB par habitant, ($ PPA internationaux consta...",2008,2008,I$,1880.800,X,Sources internationales sûres,NaN
761,FS,Données de la sécurité alimentaire,181,Zimbabwe,6126,Valeur,22013,"PIB par habitant, ($ PPA internationaux consta...",2018,2018,I$,3341.700,X,Sources internationales sûres,NaN
762,FS,Données de la sécurité alimentaire,181,Zimbabwe,6125,Valeur,21032,Stabilité politique et absence de violence/ter...,2008,2008,indice,-1.210,X,Sources internationales sûres,NaN


In [10]:
# Table pivot
data = data.pivot_table(index='Zone', values='Valeur', columns=['Élément', 'Produit', 'Année']).reset_index()
data.head(2)

Élément            Zone Disponibilité alimentaire (Kcal/personne/jour)  \
Produit                                               Produits Animaux   
Année                                                             2018   
0           Afghanistan                                          189.0   
1        Afrique du Sud                                          468.0   

Élément                \
Produit Total General   
Année            2018   
0              2270.0   
1              2900.0   

Élément Disponibilité de protéines en quantité (g/personne/jour)  \
Produit                                         Produits Animaux   
Année                                                       2018   
0                                                    10.69         
1                                                    35.17         

Élément               Exportations - Quantité Importations - Quantité  \
Produit Total General     Viande de Volailles     Viande de Volailles   
Année            2018                    2018                    2018   
0               57.62                     0.0                    25.0   
1               79.95                    48.0                   515.0   

Élément      Population totale                     Production  \
Produit Population-Estimations            Viande de Volailles   
Année                     2008       2018                2018   
0                    27722.276  37171.921                29.0   
1                    49779.471  57792.518              1762.0   

Élément                                                     Valeur           \
Produit PIB par habitant, ($ PPA internationaux constants de 2011)            
Année                                                         2008     2018   
0                                                   1484.1           2033.8   
1                                                  12628.2          12627.9   

Élément                                                                       
Produit Stabilité politique et absence de violence/terrorisme (indice)        
Année                                                             2008  2018  
0                                                    -2.69             -2.75  
1                                                     0.05             -0.22

In [11]:
# Abbréviations du multiIndex
export, imports, production, chicken, kcal, prot, pop, pop2, animal, total, secu, pib, stable = 'Exportations - Quantité', 'Importations - Quantité', 'Production', 'Viande de Volailles', 'Disponibilité alimentaire (Kcal/personne/jour)', 'Disponibilité de protéines en quantité (g/personne/jour)', 'Population totale', 'Population-Estimations', 'Produits Animaux', 'Total General', 'Valeur', 'PIB par habitant, ($ PPA internationaux constants de 2011)', 'Stabilité politique et absence de violence/terrorisme (indice)'
annee_a, annee_b = 2008, 2018


In [12]:

# Nouvelles variables
df = pd.DataFrame()
df['pays'] = data['Zone']
df['dispo_calories'] = data[kcal][total][annee_b]
df['dispo_proteines'] = data[prot][total][annee_b]
df['ratio_proteines_animales'] = data[prot][animal][annee_b] / data[prot][total][annee_b]
df['population'] = data[pop][pop2][annee_b] / 1000 # Millions d'habitants
df['population_croissance'] = data[pop][pop2][annee_b] / data[pop][pop2][annee_a]
df['pib'] = data[secu][pib][annee_b]
df['ppa'] = df['pib'] / (df['population'] * 10**6) # PIB / habitant
df['pib_croissance'] = data[secu][pib][annee_b] / data[secu][pib][annee_a]
df['poulet_import-export'] = data[imports][chicken][annee_b] / data[export][chicken][annee_b]
df

,pays,dispo_calories,dispo_proteines,ratio_proteines_animales,population,population_croissance,pib,ppa,pib_croissance,poulet_import-export
0,Afghanistan,2270.0,57.62,0.185526,37.171921,1.340868,2033.8,0.000055,1.370393,inf
1,Afrique du Sud,2900.0,79.95,0.439900,57.792518,1.160971,12627.9,0.000219,0.999976,10.729167
2,Albanie,3296.0,112.89,0.539109,2.882740,0.960056,13323.8,0.004622,1.344128,NaN
3,Algérie,3382.0,90.29,0.269133,42.228408,1.215885,11642.2,0.000276,1.078291,NaN
4,Allemagne,3556.0,105.50,0.610047,83.124418,1.025395,53463.4,0.000643,1.123818,1.360937
...,...,...,...,...,...,...,...,...,...,...
233,Îles Salomon,2405.0,52.71,0.290837,0.652857,1.296998,2697.6,0.004132,1.174453,inf
234,Îles Turques-et-Caïques,NaN,NaN,NaN,0.037665,1.207134,NaN,NaN,NaN,NaN
235,Îles Vierges américaines,NaN,NaN,NaN,0.104680,0.980554,NaN,NaN,NaN,NaN
236,Îles Vierges britanniques,NaN,NaN,NaN,0.029802,1.141970,NaN,NaN,NaN,NaN


In [13]:
df.describe(include='all')


/Users/rowaidakanaan/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/function_base.py:3961: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


,pays,dispo_calories,dispo_proteines,ratio_proteines_animales,population,population_croissance,pib,ppa,pib_croissance,poulet_import-export
count,238,180.000000,180.000000,180.000000,237.000000,230.000000,187.000000,187.000000,185.000000,131.000000
unique,238,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,Ouganda,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,2880.883333,81.819222,0.428339,38.357579,1.141948,21162.656150,0.039461,1.202973,inf
std,NaN,450.581242,20.986633,0.159945,161.816211,0.154193,22474.001096,0.160915,0.255701,NaN
min,NaN,1775.000000,26.050000,0.064112,0.000801,0.820026,761.500000,0.000005,0.494212,0.000000
25%,NaN,2587.000000,65.677500,0.290575,0.385637,1.040005,4884.350000,0.000258,1.041584,1.324333
50%,NaN,2879.500000,81.985000,0.440954,5.337962,1.118632,13323.800000,0.001667,1.165188,10.500000
75%,NaN,3275.500000,97.200000,0.562813,19.751466,1.232768,29593.400000,0.010927,1.328564,NaN


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238 entries, 0 to 237
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   pays                      238 non-null    object 
 1   dispo_calories            180 non-null    float64
 2   dispo_proteines           180 non-null    float64
 3   ratio_proteines_animales  180 non-null    float64
 4   population                237 non-null    float64
 5   population_croissance     230 non-null    float64
 6   pib                       187 non-null    float64
 7   ppa                       187 non-null    float64
 8   pib_croissance            185 non-null    float64
 9   poulet_import-export      131 non-null    float64
dtypes: float64(9), object(1)
memory usage: 18.7+ KB


In [15]:
print(df[~df.index.isin(df.dropna().index)]['pays'].to_list())

['Albanie', 'Algérie', 'Andorre', 'Anguilla', 'Antilles néerlandaises (ex)', 'Aruba', 'Bahreïn', 'Bangladesh', 'Belize', 'Bermudes', 'Bhoutan', 'Bonaire, Saint-Eustache et Saba', 'Brunéi Darussalam', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cameroun', 'Chine, Taiwan Province de', 'Chine, continentale', 'Comores', 'Cuba', 'Curaçao', 'Djibouti', 'Dominique', 'Gabon', 'Gambie', 'Gibraltar', 'Grenade', 'Groenland', 'Guadeloupe', 'Guam', 'Guinée', 'Guinée équatoriale', 'Guinée-Bissau', 'Guyane française', 'Haïti', 'Indonésie', 'Kiribati', 'Libye', 'Liechtenstein', 'Madagascar', 'Malawi', 'Maldives', 'Mali', 'Martinique', 'Mauritanie', 'Mayotte', 'Micronésie (États fédérés de)', 'Monaco', 'Montserrat', 'Nauru', 'Nioué', 'Nouvelle-Calédonie', 'Népal', 'Palaos', 'Palestine', 'Polynésie française', 'Porto Rico', 'Qatar', 'Rwanda', 'République arabe syrienne', 'République centrafricaine', 'République populaire démocratique de Corée', 'Réunion', 'Sahara occidental', 'Saint-Barthélemy', 'Saint-Ki

In [16]:
df = df.replace([np.inf, -np.inf], np.nan).dropna().reset_index().drop(['index'], axis=1)
df.style.background_gradient(cmap='BrBG')

,pays,dispo_calories,dispo_proteines,ratio_proteines_animales,population,population_croissance,pib,ppa,pib_croissance,poulet_import-export
0,Afrique du Sud,2900.000000,79.950000,0.439900,57.792518,1.160971,12627.900000,0.000219,0.999976,10.729167
1,Allemagne,3556.000000,105.500000,0.610047,83.124418,1.025395,53463.400000,0.000643,1.123818,1.360937
2,Arabie saoudite,3308.000000,86.580000,0.376184,33.702756,1.301841,47592.800000,0.001412,1.049684,18.285714
3,Argentine,3307.000000,108.490000,0.643746,44.361150,1.106811,22759.400000,0.000513,0.985106,0.078788
4,Australie,3394.000000,107.510000,0.672310,24.898152,1.167158,49151.700000,0.001974,1.096270,0.410256
5,Autriche,3699.000000,109.710000,0.601404,8.891388,1.065918,55297.600000,0.006219,1.041584,1.373494
6,Azerbaïdjan,3145.000000,94.620000,0.354893,9.949537,1.127826,14209.600000,0.001428,1.101246,35.000000
7,Belgique,3774.000000,98.080000,0.579833,11.482178,1.065260,51111.000000,0.004451,1.055520,0.474178
8,Bolivie (État plurinational de),2476.000000,74.520000,0.478798,11.353142,1.167844,8655.500000,0.000762,1.362813,0.000000
9,Bosnie-Herzégovine,3317.000000,99.290000,0.373351,3.323925,0.885372,14386.700000,0.004328,1.303734,1.500000


In [17]:
import geopandas as gpd


In [18]:
#conda install geoplot -c conda-forge


In [19]:
import geopandas as gpd


In [20]:
#pip uninstall pyshp

In [22]:
conda install geoplot -c conda-forge

Solving environment: done

## Package Plan ##

  environment location: /Users/rowaidakanaan/opt/anaconda3

  added / updated specs:
    - geoplot


The following NEW packages will be INSTALLED:

  affine             conda-forge/noarch::affine-2.3.0-py_0
  boost-cpp          conda-forge/osx-64::boost-cpp-1.74.0-hdbf7018_7
  cached-property    conda-forge/noarch::cached-property-1.5.2-hd8ed1ab_1
  cached_property    conda-forge/noarch::cached_property-1.5.2-pyha770c72_1
  cartopy            conda-forge/osx-64::cartopy-0.20.2-py38he8a5f01_2
  cfitsio            conda-forge/osx-64::cfitsio-4.0.0-hb20e66c_0
  contextily         conda-forge/noarch::contextily-1.2.0-pyhd8ed1ab_0
  font-ttf-dejavu-s~ conda-forge/noarch::font-ttf-dejavu-sans-mono-2.37-hab24e00_0
  font-ttf-inconsol~ conda-forge/noarch::font-ttf-inconsolata-3.000-h77eed37_0
  font-ttf-source-c~ conda-forge/noarch::font-ttf-source-code-pro-2.038-h77eed37_0
  font-ttf-ubuntu    conda-forge/noarch::font-ttf-ubuntu-0.83-hab24e00_0
 

Preparing transaction: done
Verifying transaction: failed

CondaVerificationError: The package for pytables located at /Users/rowaidakanaan/opt/anaconda3/pkgs/pytables-3.7.0-py38h5215406_0
appears to be corrupted. The path 'lib/python3.8/site-packages/tables-3.7.0.dist-info/direct_url.json'
specified in the package manifest cannot be found.

CondaVerificationError: The package for pytables located at /Users/rowaidakanaan/opt/anaconda3/pkgs/pytables-3.7.0-py38h5215406_0
appears to be corrupted. The path 'lib/python3.8/site-packages/tables/__init__.py'
specified in the package manifest cannot be found.

CondaVerificationError: The package for pytables located at /Users/rowaidakanaan/opt/anaconda3/pkgs/pytables-3.7.0-py38h5215406_0
appears to be corrupted. The path 'lib/python3.8/site-packages/tables/__pycache__/array.cpython-38.pyc'
specified in the package manifest cannot be found.

CondaVerificationError: The package for pytables located at /Users/rowaidakanaan/opt/anaconda3/pkgs/pytab


Note: you may need to restart the kernel to use updated packages.


In [1]:
import geopandas as gpd
import geoplot

ImportError: cannot import name 'GeoAxesSubplot' from 'cartopy.mpl.geoaxes' (/Users/rowaidakanaan/opt/anaconda3/lib/python3.8/site-packages/cartopy/mpl/geoaxes.py)